In [3]:
import pandas as pd
import numpy as np

In [4]:
def conv(x):
    temp = x.rstrip('%')
    temp = temp.replace(',', '.')
    return float(temp)

# geht nur ohne NAs

In [5]:
muenchen_nord = pd.read_csv('../../Daten/muenchen_nord.txt', sep = ';')
maxvorstadt = pd.read_csv('../../Daten/Stadtbezirke/maxvorstadt.txt', sep = ';')
schwabing_west = pd.read_csv('../../Daten/Stadtbezirke/schwabing_west.txt', sep = ';')
freimann = pd.read_csv('../../Daten/Stadtbezirke/schwabing_freimann.txt', sep = ';')
milbertshofen = pd.read_csv('../../Daten/Stadtbezirke/milbertshofen_amhart.txt', sep = ';')
moosach = pd.read_csv('../../Daten/Stadtbezirke/moosach.txt', sep = ';')
hasenbergl = pd.read_csv('../../Daten/Stadtbezirke/feldmoching_hasenbergl.txt', sep = ';')

## Erst- und Zweitstimmen `17

In [7]:
Erst = pd.read_csv('../../Daten/erststimmen.csv', sep = ';', converters = {'Anteil SPD': conv, 'Anteil CSU': conv,'Anteil GRÜNE': conv, 'Anteil FDP': conv,
                                                                            'Anteil AfD': conv, 'Anteil DIE LINKE': conv,
                                                                            'Anteil FREIE WÄHLER': conv, 'Anteil ÖDP': conv, 'Anteil BP': conv, 'Anteil BüSo': conv})
Zweit = pd.read_csv('../../Daten/zweitstimmen.csv', sep = ';', converters = {'Anteil SPD': conv, 'Anteil CSU': conv,'Anteil GRÜNE': conv, 'Anteil FDP': conv,
                                                                            'Anteil AfD': conv, 'Anteil DIE LINKE': conv,
                                                                            'Anteil FREIE WÄHLER': conv, 'Anteil ÖDP': conv, 'Anteil BP': conv, 'Anteil BüSo': conv,
                                                                            'Anteil PIRATEN': conv, 'Anteil NPD': conv,
                                                                            'Anteil Tierschutzpartei': conv, 'Anteil BGE': conv, 'Anteil DiB': conv,
                                                                            'Anteil DKP': conv, 'Anteil DM': conv, })

#nicht_relevant_erst = ['Gebietsartschlüssel', 'Stimmbezirksart', 'Wahlberechtigte ohne Wahlschein', 'Wähler mit Wahlschein',
#                 'Ungültige Stimmen', 'Anzahl SPD', 'Anzahl CSU', 'Anzahl GRÜNE', 'Anzahl DIE LINKE', 'Anzahl FREIE WÄHLER',
#                 'Anzahl ÖDP', 'Anzahl BP', 'Anzahl MLPD', 'Anzahl BüSo', 'Anzahl Die PARTEI', 'Anzahl BB', 'Anteil BB']

relevant_erst = ['Gebietsnummer', 'Anteil CSU', 'Anteil SPD', 'Anteil GRÜNE', 
                'Anteil FDP', 'Anteil AfD', 'Anteil DIE LINKE']

relevant_zweit = ['Gebietsnummer', 'Anteil CSU', 'Anteil SPD', 'Anteil GRÜNE', 
                'Anteil FDP', 'Anteil AfD', 'Anteil DIE LINKE']

Erst = Erst[relevant_erst]
Zweit = Zweit[relevant_zweit]


muenchen_nord = pd.read_csv('../../Daten/muenchen_nord.txt', sep = ';')
gebietsnummern = np.asarray(muenchen_nord['Name'])

Erst = Erst[Erst['Gebietsnummer'].isin(gebietsnummern)]
Zweit = Zweit[Zweit['Gebietsnummer'].isin(gebietsnummern)]

Erst = Erst.rename(columns = {'Anteil SPD': 'SPD', 'Anteil CSU': 'CSU','Anteil GRÜNE': 'GRÜNE', 'Anteil FDP': 'FDP',
                                                                        'Anteil AfD': 'AfD', 'Anteil DIE LINKE': 'DIE LINKE'})
Zweit = Zweit.rename(columns = {'Anteil SPD': 'SPD', 'Anteil CSU': 'CSU','Anteil GRÜNE': 'GRÜNE', 'Anteil FDP': 'FDP',
                                                                        'Anteil AfD': 'AfD', 'Anteil DIE LINKE': 'DIE LINKE'})

In [8]:
Erst.to_csv('../../Daten/erststimmen_aufbereitet.csv', index = False)
Zweit.to_csv('../../Daten/zweitstimmen_aufbereitet.csv', index = False)

## Strukturindikatoren `17

In [159]:
struktur = pd.read_csv('../../Daten/strukturindikatoren.txt', sep = ';')
arbeitslose = pd.read_csv('../../Daten/arbeitslosendichte_17.txt', sep = ';')
alg = pd.read_csv('../../Daten/alg_empfängerdichte_17.txt', sep = ';')

In [160]:
columns = struktur.columns[2::]
for column in columns:
    struktur[column] = struktur[column].apply(lambda x: conv(x))

In [161]:
struktur.drop(struktur.tail(1).index,inplace=True) # drop last n rows
struktur['Wahlbezirk'] = struktur['Wahlbezirk'].apply(lambda x: conv(x))

In [162]:
struktur = struktur[struktur['Wahlbezirk'].isin(gebietsnummern)]
struktur['Wahlbezirk'] = struktur['Wahlbezirk'].apply(lambda x: int(x))

In [183]:
struktur = struktur.reset_index(drop = True)
struktur = struktur.rename(columns = {'Wahlbezirk': 'Gebietsnummer'})

In [165]:
# SV-Beschäftigtendichte
struktur['SV-Beschäftigtendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '03 Maxvorstadt', 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '04 Schwabing - West', 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '10 Moosach', 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '11 Milbertshofen -  Am Hart', 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '12 Schwabing -  Freimann', 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '24 Feldmoching - Hasenbergl', 'SV-Beschäftigtendichte'].iloc[0]

# Arbeitslosendichte
struktur['Arbeitslosendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '03 Maxvorstadt', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '04 Schwabing - West', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '10 Moosach', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '11 Milbertshofen -  Am Hart', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '12 Schwabing -  Freimann', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '24 Feldmoching - Hasenbergl', 'Arbeitslosendichte'].iloc[0]

# ALG II - Empfängerdichte
struktur['ALG II - Empfängerdichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'ALG II - Empfängerdichte'] = alg.loc[
    alg['Stadtbezirk'] == '03 Maxvorstadt', 'ALG II - Empfängerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'ALG II - Empfängerdichte'] = alg.loc[
    alg['Stadtbezirk'] == '04 Schwabing - West', 'ALG II - Empfängerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'ALG II - Empfängerdichte'] = alg.loc[
    alg['Stadtbezirk'] == '10 Moosach', 'ALG II - Empfängerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'ALG II - Empfängerdichte'] = alg.loc[
    alg['Stadtbezirk'] == '11 Milbertshofen -  Am Hart', 'ALG II - Empfängerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'ALG II - Empfängerdichte'] = alg.loc[
    alg['Stadtbezirk'] == '12 Schwabing -  Freimann', 'ALG II - Empfängerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'ALG II - Empfängerdichte'] = alg.loc[
    alg['Stadtbezirk'] == '24 Feldmoching - Hasenbergl', 'ALG II - Empfängerdichte'].iloc[0]

In [128]:
struktur.to_csv('../../Daten/strukturindikatoren.csv')

## Strukturindikatoren `19

In [173]:
struktur = pd.read_csv('../../Daten/strukturindikatoren_19_raw.csv', sep = ';')
struktur = struktur.drop(columns = ['SV-Beschäftigtendichte', 'Arbeitslosendichte',
                                   'Erwerbsfähige Leistungsberechtigtendichte', 'Gebietsartschlüssel'])
struktur = struktur.loc[struktur['Gebietsnummer'].isin(np.array(muenchen_nord.Name))]
arbeitslose = pd.read_csv('../../Daten/stadtbezirke_struktur_19.csv', sep = ';')
arbeitslose = arbeitslose.dropna()
arbeitslose['SV-Beschäftigtendichte'] = arbeitslose['SV-Beschäftigtendichte'].apply(lambda x: conv(x))
arbeitslose['Arbeitslosendichte'] = arbeitslose['Arbeitslosendichte'].apply(lambda x: conv(x))
arbeitslose['Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose[
    'Erwerbsfähige Leistungsberechtigtendichte'].apply(lambda x: conv(x))
arbeitslose = arbeitslose.rename(columns = {'Gebietsnummer': 'Stadtbezirk'})

In [174]:
columns = struktur.columns[2::]
for column in columns:
    struktur[column] = struktur[column].apply(lambda x: conv(x))

In [175]:
arbeitslose['Stadtbezirk'] = arbeitslose['Stadtbezirk'].apply(lambda x: int(x))

In [179]:
# SV-Beschäftigtendichte
struktur['SV-Beschäftigtendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 3, 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 4, 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 10, 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 11, 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 12, 'SV-Beschäftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 24, 'SV-Beschäftigtendichte'].iloc[0]

# Arbeitslosendichte
struktur['Arbeitslosendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 3, 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 4, 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 10, 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 11, 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 12, 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 24, 'Arbeitslosendichte'].iloc[0]

# Erwerbsfähige Leistungsberechtigtendichte
struktur['Erwerbsfähige Leistungsberechtigtendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 3, 'Erwerbsfähige Leistungsberechtigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 4, 'Erwerbsfähige Leistungsberechtigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 10, 'Erwerbsfähige Leistungsberechtigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 11, 'Erwerbsfähige Leistungsberechtigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 12, 'Erwerbsfähige Leistungsberechtigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'Erwerbsfähige Leistungsberechtigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == 24, 'Erwerbsfähige Leistungsberechtigtendichte'].iloc[0]

In [182]:
struktur.to_csv('../../Daten/strukturindikatoren_19.csv')

## Strukturindikatoren `14

In [228]:
struktur = pd.read_csv('../../Daten/strukturindikatoren_14.txt', sep = ';')
struktur = struktur.rename(columns = {'Wahlbezirk': 'Gebietsnummer'})
struktur = struktur.loc[struktur['Gebietsnummer'].isin(np.array(muenchen_nord.Name))]
arbeitslose = pd.read_csv('../../Daten/stadtbezirke_struktur_14.txt', sep = ';',
                         converters = {'SV-Beschäftigtendichte': conv,
                                       'Arbeitslosendichte': conv, 'ALG II- Empfaengerdichte': conv})
arbeitslose = arbeitslose.rename(columns = {'Bezirk': 'Stadtbezirk'})

In [229]:
columns = struktur.columns[2::]
for column in columns:
    struktur[column] = struktur[column].apply(lambda x: conv(x))

In [230]:
# SV-Beschäftigtendichte
struktur['SV-Beschäftigtendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '03 Maxvorstadt', 'SV-Beschaeftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '04 Schwabing West', 'SV-Beschaeftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '10 Moosach', 'SV-Beschaeftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '11 Milbertshofen -  Am Hart', 'SV-Beschaeftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '12 Schwabing -  Freimann', 'SV-Beschaeftigtendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'SV-Beschäftigtendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '24 Feldmoching - Hasenbergl', 'SV-Beschaeftigtendichte'].iloc[0]

# Arbeitslosendichte
struktur['Arbeitslosendichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '03 Maxvorstadt', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '04 Schwabing West', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '10 Moosach', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '11 Milbertshofen -  Am Hart', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '12 Schwabing -  Freimann', 'Arbeitslosendichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'Arbeitslosendichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '24 Feldmoching - Hasenbergl', 'Arbeitslosendichte'].iloc[0]

# ALG II - Empfängerdichte
struktur['ALG II - Empfängerdichte'] = 0
struktur.loc[struktur['Gebietsnummer'].isin(np.array(maxvorstadt.Name)), 
             'ALG II - Empfängerdichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '03 Maxvorstadt', 'ALG II- Empfaengerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(schwabing_west.Name)), 
             'ALG II - Empfängerdichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '04 Schwabing West', 'ALG II- Empfaengerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(moosach.Name)), 
             'ALG II - Empfängerdichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '10 Moosach', 'ALG II- Empfaengerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(milbertshofen.Name)), 
             'ALG II - Empfängerdichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '11 Milbertshofen -  Am Hart', 'ALG II- Empfaengerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(freimann.Name)), 
             'ALG II - Empfängerdichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '12 Schwabing -  Freimann', 'ALG II- Empfaengerdichte'].iloc[0]
struktur.loc[struktur['Gebietsnummer'].isin(np.array(hasenbergl.Name)), 
             'ALG II - Empfängerdichte'] = arbeitslose.loc[
    arbeitslose['Stadtbezirk'] == '24 Feldmoching - Hasenbergl', 'ALG II- Empfaengerdichte'].iloc[0]

In [232]:
struktur.to_csv('../../Daten/strukturindikatoren_14.csv')